<a href="https://colab.research.google.com/github/jazkre/01RAD/blob/main/assignments/2022/01RAD2022_assignement02_Kreanova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2. zápočtová úloha z 01RAD


> > Jazmína Kreanová





## Popis úlohy

Datový soubor `Boston` je obsažen v balíku `MASS` a lze použít rovnou po načtení příslušné knihovny.

In [ ]:
knitr::opts_chunk$set(echo = FALSE)
list_of_packages <- c("tidyverse", "MASS","GGally","moderndive", "dplyr","car","lmtest","nortest")
missing_packages <- list_of_packages[!(list_of_packages %in% installed.packages()[,"Package"])]
missing_packages
if(length(missing_packages)) install.packages(missing_packages)
lapply(list_of_packages, library, character.only = TRUE)
#library(tidyverse)
#library(GGally)
#library(knitr)
#library(MASS)

In [ ]:
#install.packages("car")
#library(car)
fig <- function(width, heigth){
 options(repr.plot.width = width, repr.plot.height = heigth)
 }
 options(jupyter.plot_scale=2)

In [ ]:
head(Boston)

Obsahuje celkem 506 záznamů z obcí v předměstí města Boston, MA, USA a data pocházejí ze studie v roce 1978.Viz Harrison, D. and Rubinfeld, D.L. (1978) Hedonic prices and the demand for clean air. J. Environ. Economics and Management 5, 81–102.

Základní charakteristiky ohledně jednotlivých proměnných získáte pomocí funkcí `str(Boston)} a `summary(Boston)}.

Data celkem obsahují 14 proměnných, přičemž naším cílem je prozkoumat vliv 13 z nich na cenu nemovitostí `medv`. Přičemž anglický popis jednotlivých proměnných (sloupců) je následující:

Feature & Description 

`crim` per capita crime rate by town  \\

`zn`: proportion of residential land zoned for lots over 25,000 sq.ft  \\

`indus`: proportion of non-retail business acres per town  \\

`chas`: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)  

`nox`: nitrogen oxides concentration (parts per 10 million)  \\

`rm`: average number of rooms per dwelling  \\

`age`: proportion of owner-occupied units built prior to 1940  \\

`dis`: weighted mean of distances to five Boston employment centres  \\

`rad`: index of accessibility to radial highways  \\

`tax`: full-value property-tax rate per \$10,000  \\

`ptratio`: pupil-teacher ratio by town  \\

`black`:  $1000(B_k - 0.63)^2$ where $B_k$ is the proportion of blacks by town  \\

`lstat`: lower status of the population (percent)  \\

`medv`: median value of owner-occupied homes in \$1000s  \\



## Podmínky a body

Úkol i protokol vypracujte samostatně. Pokud na řešení nějaké úlohy budete přesto s někým spolupracovat, radit se, nezapomeňte to u odpovědi na danou otázku uvést. Tato zápočtová úloha obsahuje 10 otázek po 1 bodu. Celkem za 3 zápočtové úlohy bude možné získat 30 bodů, přičemž pro získání zápočtu je potřeba více jak 20. Další dodatečné body mohu případně individuálně udělit za extra práci na mini domácích úkolech nebo za aktivitu v hodině.

## Odevzdání

Protokol ve formátu pdf (včetně příslušného Rmd souboru), nebo jak jupyter NB (ideálně s odkazem na Colab) odevzdejte prostřednictvím MS Teams, nejpozději do půlnoci  14. 12. 2022 (tj. za 3 týdny).

## Příprava dat:

* Otázka 01

  Z dat vyfilitrujte jen pozorování, kde proměnná `chas` je rovna 0, proměnná rad je menší než 20 a odezva `medv` neobsahuje opakující se maximální hodnoty vzniklé nejspíše zaokrouhlením. Zkontrolujte, že výsledný datset neobsahuje chybějící hodnoty a vykreslete scatterplot pro proměnné `indus` a `medv`.

In [ ]:
summary(Boston)



In [ ]:
#table(factor(Boston$medv))

Našou úlohou je vyfiltrovať dataset tak, aby zostali pozorovania s $`chas=0`, `rad < 20`$ a aby dataset neobsahoval opakujúce sa maximálne hodnoty v odozve `medv`.

Vypíšeme si najprv pozorovania s opakujúcou sa hodnotou maxima v kýženej odozve `medv`. To činí 8 rôznych pozorovaní, pričom nepovažujeme za vhodné vybrať jedno pozorovanie, ktoré by sme do datasetu začlenili.

V ďalšom kroku teda vyfiltrujeme dáta tak, že nebydú obsahovať žiadne pozorovanie, ktoré v premennej `medv` nadobúda maximum. a overíme, že v datasete nemáme chýbajúce hodnoty. Nový dataset obsahuje 339 pozorovaní.

In [ ]:
subset(Boston, Boston$chas==0 & Boston$rad<20 & Boston$medv==max(Boston$medv))
#dim(Boston_mx)

In [ ]:
Boston_data<-subset(Boston, Boston$chas==0 & Boston$rad<20 & Boston$medv != max(Boston$medv))

In [ ]:
head(Boston_data)
dim(Boston_data)

In [ ]:
Boston_data %>% 
  filter(if_any(everything(), is.na))

Vykreslíme scatterplot pre odozvu `medv` a vysvetľujúcu premennú `indus`. Podľa vykresleného scatterplotu nepozorujeme viditeľnú lineárnu závislosť mediánovej ceny nehnuteľností na podieli nemaloobchodného podnikania. Predpokladáme, že cena teda závisí aj na iných premenných, respektíve premenná `indus` môže byť závislá.

In [ ]:
ggplot(Boston_data,aes(x=indus,y=medv))+
  geom_point(col="plum3")+
  labs(x= "Proportion of non-retail business acres per town", y="Median value of owner-occupied homes in $100s")

## Regresní model závislosti mediánu ceny nemovitosti na zastoupení ne-maloobchodního podnikání v daném místě: 


* Otázka 2

  Sestavte jednoduchý regresní model a na jeho základech zjistěte zdali proměnná `indus`  ovlivňuje median ceny nemovitostí určených k bydlení `medv`. Pokud ano, o kolik je průměr mediánů cen nemovitostí nižší/vyšší při vzrůstu zastoupení nemaloobchodního podnikání o 5 jednotek? 

Zostavíme jednoduchý regresný model závislosti ceny nehnuteľností od podielu podnikania v danej oblasti. Volíme model s interceptom, pretože predpokladáme, že nehnuteľnosť bude mať nenulovú cenu aj v oblasti bez dostatočnej občianskej vybavenosti, navyše z predošlej úvahy v otázke jedna predpokladáme aj iné závislosti.



In [ ]:
model = lm(medv ~ indus, data = Boston_data)
summary(model)

In [ ]:
model$coefficients[2]*5

Odhadli sme model: $Y= 28.434 -0.536\cdot X$,

kde nám intercept aj premenná `indus` vyšli ako štatisticky významné, *p-hodnota* oboch parametrov vyšla veľmi malá ($2 \cdot 10^{-16}$). Hodnota charakteristiky $R^2=0.2181$ je pomerne nízka (čo sme v predošlých úvahách očakávali).

Na základe nášho modelu dôjde pri náraste nemaloobchodného podnikania o 5 jednotiek k poklesu (koeficient má zápornú hodnotu) cien v priemere o 2679.92 \$.



* Otázka 3

  Vyzkoušejte model s mocninou a logaritmickou transformací odezvy. Pro výběr mocniné transformace vykreslete optimální  log-věrohodnostní profil u Box-Coxovy transformace a porovnejte navrženou transformaci s provedenou logaritmickou.

Vykreslíme ploty reziduí našeho modelu. Z grafov Resuduals vs. Fitted a Scale-Location pozorujeme, že reziduá sa v rozptyle rozbiehajú, teda dochádza k porušeniu podmienku homoskedasticity, naviac Q-Q plot indikuje ťažké chvosty, čo nás navádza na transformáciu veličín. 

In [ ]:
#cv8,9
par(mfrow = c(2, 2))
plot(model, col="plum3",pch=20)

**Mocninná transformácia**
Box-Coxova transformácia:
$$g(Y + a) = ln(Y + a) \ \text{if} \  \ λ = 0,$$
$$g(Y + a) = \frac{(Y + a)^{\lambda} - 1}{\lambda} \ \ \text{if} \   λ \neq 0,$$


Chceme transformovať dáta pomocou Box.Coxovej transformácie. Ako najvhodnejší exponent nám vyšlo $\lambda=-0.16$, čo spadá do intervalu spoľahlivosti $0$ a teda vhodné by bolo použiť logaritmickú transformáciu.

Našou úlohou v tejto časti je porovnať mocninnú a logaritmickú transformáciu, preto nebudeme uvažovať $\lambda=0$.

In [ ]:
bc     <- boxcox(model, lambda = seq(-1,1 , 1/100))
lambda <- bc$x[which.max(bc$y)]
print("lambda:")
lambda
print("CI:")
c(min(bc$x[bc$y > max(bc$y) - 1/2 * qchisq(.95,1)]),max(bc$x[bc$y > max(bc$y) - 1/2 * qchisq(.95,1)]))

Aplikujeme mocninnú transformáciu pre naše λ:

In [ ]:
# Power transform
Boston_bcT = Boston_data
Boston_bcT$medv_pwr = (Boston_bcT$medv^lambda-1)/lambda
model_pwr = lm(medv_pwr~indus, data = Boston_bcT)
summary(model_pwr)

In [ ]:
par(mfrow = c(2, 2))
plot(model_pwr, col="plum3",pch=20)

**Logaritmická transformácia**
$$\ln(y)= 𝐗 \cdot \beta + \epsilon$$

$$Y= e^{𝐗 \cdot \beta + \epsilon}=e^{𝐗 \cdot \beta} e^\epsilon$$

In [ ]:
model_ln = lm(log(medv) ~ indus, Boston_data)
summary(model_ln)

In [ ]:
par(mfrow = c(2, 2))
plot(model_ln, col="plum3",pch=20)

**Porovnanie**

In [ ]:
#summary(model)
#summary(model_ln)
#summary(model_pwr)


Chceme porovnať naše modely.

Pôvodný lineárny model mal hodnotu charakteristiky $R^2=0.2181$ a smerodajnú odchylku reziduí $sd_e=6.447$. S použitím mocninnej transformácie sa hodnota zvýšila na $R^2=0.2657$, smerodajná odchylka poklesla na $sd_e=0.1523$ a vrámci logaritmickej transformácie sme obdržali hodnotu $R^2=0.2607$, čo je nižsie ako pre mocninnú, ale vyššie než pre netransformovaný model a a $sd_e=0.2519$ opäť trochu horšie než pre mocninný model a markantne lepšie než pre netransformovaný model.

Ďalej vyobrazíme Q-Q ploty pre jednotlivé modely. Môžeme pozorovať, že oba transformované modely grafy indikujú normalitu dát.



In [ ]:
#fig(12,12)
op <- par(mfrow=c(3,2))
plot(model, sub = "Bez tranfsormacie", which = 1, pch = 20, col = "plum3", lwd = 2)
plot(Boston_data$indus, sqrt(abs(rstandard(model))), sub = "Bez tranfsormacie", pch = 20, col = "plum3", bg = "royalblue4",
     xlab = "Indus (1000$)", ylab = as.expression(substitute(sqrt(abs(yL)), list(yL = as.name("Standardized residuals")))),
     main = "Scale-Location (Indus)")
lines(lowess(Boston_data$indus, sqrt(abs(rstandard(model)))), col = "red3", lwd = 2)
abline(lm(sqrt(abs(rstandard(model)))~Boston_data$indus), col = "mediumseagreen", lwd = 2)
plot(model_ln, sub = "Logaritmická transf.", which = 1, pch = 20, col = "plum3", lwd = 2)
plot(Boston_data$indus, sqrt(abs(rstandard(model_ln))), pch = 20, col = "plum3", bg = "royalblue4",
     xlab = "Indus (1000$)", ylab = as.expression(substitute(sqrt(abs(yL)), list(yL = as.name("Standardized residuals")))),
     sub = "Logaritmická transf.",main = "Scale-Location (Indus)")
lines(lowess(Boston_data$indus, sqrt(abs(rstandard(model_ln)))), col = "red3", lwd = 2)
abline(lm(sqrt(abs(rstandard(model_ln)))~Boston_data$indus), col = "mediumseagreen", lwd = 2)

plot(model_pwr, sub = "Mocninná transf.", which = 1, pch = 20, col = "plum3", lwd = 2)
plot(Boston_data$indus, sqrt(abs(rstandard(model_pwr))), pch = 20, col = "plum3", bg = "royalblue4",
     xlab = "Indus (1000$)", ylab = as.expression(substitute(sqrt(abs(yL)), list(yL = as.name("Standardized residuals")))),
     sub = "Mocninná transf.",main = "Scale-Location (Indus)")
lines(lowess(Boston_data$indus, sqrt(abs(rstandard(model_pwr)))), col = "red3", lwd = 2)
abline(lm(sqrt(abs(rstandard(model_pwr)))~Boston_data$indus), col = "mediumseagreen", lwd = 2)

In [ ]:
fig(12,12)
op <- par(mfrow=c(2,3))
plot(model,which=2, pch = 20, col = "plum3", lwd = 2,sub = "Bez transf.")
plot(model_ln,which=2, pch = 20, col = "plum3", lwd = 2,sub = "Logaritmická transf.")
plot(model_pwr,which=2,  pch = 20, col = "plum3", lwd = 2,sub = "Mocninná transf.")
hist(resid(model),breaks=30, col = "plum3",sub = "Bez transf.")
hist(resid(model_ln),breaks=30, col = "plum3",sub = "Logaritmická transf.")
hist(resid(model_pwr),breaks=30, col = "plum3",sub = "Mocninná transf.")

In [ ]:
# compare models with and without log transform
op <- par(mfrow=c(3,2))
qqPlot(model, simulate=TRUE, line="quartiles")
plot(density(rstudent(model)))
qqPlot(model_ln, simulate=TRUE, line="quartiles")
plot(density(rstudent(model_ln)))
qqPlot(model_pwr, simulate=TRUE, line="quartiles")
plot(density(rstudent(model_pwr)))
par(op)

Aplikujeme príslušné testy, aby sme prípadne vyvrátili naše domnienky z grafov (podotknime, že v praxi často stačí záver na základe Q-Qplotov, či absencia kónusu reziduálneho scatterplotu).

Pre test normality využijeme Lillieforsov, Shapiro-Wilkov a Anderson-Darlingov test s levelom signifikantnosti α=5%. Pre netransformovaný model dostávame veľmi nízke p-hodnoty (pod $10^{-7}$) pre všetky testy a teda zamietame hypotézu o normalite dát. Pre logaritmickú transformáciu máme hodnoty nad 0.077, čiže nemáme dostatok dókazov na zamietnutie hypotézy o normalite dát a pre mocninnú transformáciu sú p-hodnoty o niečo vyššie a opäť nemožno hypotézu normality zamietnuť.



In [ ]:
#Normality
print("************************Orig*************************")
lillie.test(resid(model))   # Lilliefors test
shapiro.test(resid(model))      # Shapiro-Wilk test
ad.test(resid(model))
print("************************Log*************************")
lillie.test(resid(model_ln))   # Lilliefors test
shapiro.test(resid(model_ln))      # Shapiro-Wilk test
ad.test(resid(model_ln))
print("************************Power*************************")
lillie.test(resid(model_pwr))   # Lilliefors test
shapiro.test(resid(model_pwr))      # Shapiro-Wilk test
ad.test(resid(model_pwr))

Rovnako potrebujeme otestovať homoskedasticitu dát. Využijeme Breusch-Pagan test, s nutnosťou normálne rozdelených reziduí. Testujeme len transformované modely. Dostávame hodnoty 0.79 pre logaritmickú a 0.24 pre mocninnú transformáciu v dôsledku čoho pri α=5% nezamietame homoskedasticitu.

Test homoskedasticity vyšiel významnejšie pre logaritmickú transformáciu, čo spolu s úzusom z praxe pre λ z konfidenčného intervalu 0 a ľahšou interpretovateľnosťou modelu s touto transformáciou považujeme za vhodné ďalej pracovať práve s logaritmicky transformovaným modelom

In [ ]:
# for dependence on mean value (residual variance on the response expectation)
# Breusch-Pagan test statistic - library lmtest

print("************************Log*************************")
bptest(model_ln)
print("************************Power*************************")
bptest(model_pwr)

### cv_8 help

In [ ]:
# # for dependence on mean value (residual variance on the response expectation)
# # Breusch-Pagan test statistic - library lmtest
# bptest(cars_lm3)
# bptest(cars_lm3,varformula = ~fitted(cars_lm3),studentize = T)
# bptest(cars_lm3,varformula = ~fitted(cars_lm3),studentize = FALSE)
# # Score Test for Non-Constant Error Variance (Breusch-Pagan test) library:car
# ncvTest(cars_lm3)
# 
# # check dependence on covariate 
# with(cars,bptest(cars_lm3,varformula = ~speed,studentize = FALSE))
# ncvTest(cars_lm3, var.formula = ~speed)
# 
# 
# # Normality 
# 
# # shapiro.test
# # lillie.test
# # ad.test
# # library(ICSOutlier) jarque.test, anscombe.test
# 
# par(mfrow = c(1, 1))
# plot(cars_lm3, which = 2, pch = 20, col = "blue4", lwd = 2)
# library(nortest)
# #budeme používať Liliefors a Shapira
# #KS má problém ak máme opakujúce sa pozorovanie v 1 bode
# lillie.test(residuals(cars_lm3))   # Lilliefors test
# #ks.test(residuals(cars_lm3),"pnorm",0,15)
# shapiro.test(resid(cars_lm3))      # Shapiro-Wilk test
# shapiro.test(residuals(cars_lm3))  # same
# ad.test(resid(cars_lm3))
# 
# summary(cars_lm3)
# 
# # Independence (Checking uncorrelated errors)
# #chyba može závisieť na poradi merania (únava pozorovateľa...)
# #možeme čeknuť "zpoždeňí rezidua"
# # Assume that data are measured from one car - step-by-step
# # check if there is any independence (driver is tired, breaks abraded, ...)
# 
# # bgtest library(lmtest)
# bgtest(cars_lm3)
# dwtest(cars_lm3) # One-sided 
# dwtest(cars_lm3, alternative = "two.sided")  # Two-sided 
# 
# set.seed(420)
# durbinWatsonTest(cars_lm3, alternative = "positive") # One-sided 
# durbinWatsonTest(cars_lm3)  # Two-sided 

### .

* Otázka 4

  Z log transformovaného modelu vyčtěte procentuální navýšení/pokles ceny nemovitostí při změně zastoupení nemaloobchodního podnikání o 5 jednotek  (odpověď typu: 
Střední cena nemovitosti v lokalitách okolo Bostonu,  liších se podílem nemaloobchodních zón, klesá/roste zhruba o XX% na každou 1 jednotku nárůstu/poklesu podílu nemaloobchodních zón.

Jednotkovú zmenu $x_j \longrightarrow x_j + 1$ dostaneme podelením modelov:
$$\frac{E[Y_{new}]}{E[Y]}=\frac{e^{\hat{\beta_0}+\hat{\beta_1}x_1 + \dots + \hat{\beta_j}(x_j+1) + \dots +\hat{\beta_m}x_m}}{e^{\hat{\beta_0}+\hat{\beta_1}x_1 + \dots + \dots +\hat{\beta_m}x_m}}=e^{\hat{\beta_j}},$$

čo možno zapísať ako $100(\hat{\beta_j}-1)\%$ zmena priemernej odozvy s jednotkovou zmenou $x_j$.

Pre náš model dostávame, že stredná cena nehnuteľností v lokalitách okolo Bostonu, líšiacich sa s podielom nemaloobchodných zón klesá zhruba o 2.33% na každú 1 jednotku nárastu podielu nemaloobchodných zón a teda poklesne o 11.11% pri náraste zastúpenia nemaloobchodných zón o 5 jednotiek. 

In [ ]:
unname(100*(exp(model_ln$coefficients[2])-1))
unname(100*(exp(5*model_ln$coefficients[2])-1))



* Otázka 5

  Vyberte jeden z předešlých modelů (s/bez transformované odezvy) a zkuste transformovat i nezávislou proměnnou `indus`. Vyzkoušejte například po částech konstantní transformaci (odpovídající faktorizaci nezávislé proměnné), splines a polynomiální transformace (kvadratickou a kubickou). Zkuste využít informací získaných například z `crPlots(model)`. Lze některé z těchto modelů testovat mezi sebou F-testem? Pokud ano, proveďte a diskutujte.  

V ďalšej časti pokračujeme s modelom s logaritmicou transformáciou odozvy - reziduá spĺňajú požiadavku homoskedasticity, ich rozdelenie akceptujeme (test normalitu nezamietol, Q.Qplot tiež vyhovuje) ako normálne a dá sa interpretovať.

Vykreslíme  `crPlots` (component residual plot) zvoleného modelu. Vidíme, že komponenty vie sú vyvážene rozložené okolo reziduálnej priamky. Aby sme tento trend poupravili, transformujeme prediktor `indus` (logaritmicky, polynomiálne).

In [ ]:
fig(7,6)
crPlots(model_ln, col = "plum3",pch=20)


**Logaritmická transformácia prediktora**

Model s transformovaným prediktorom má $R^2=0.2921$, čo je zvýšenie oproti východziemu modelu(0.2607) a Residual standard error: 0.2465, opäť zlepšená hodnota (0.2519).
Na vykreslenom component residual plote pozorujeme zlepšenie rozmiestnenia dát okolo krivky predikcie modelu.

In [ ]:
model_ln_pred=lm(log(medv)~log(indus), data=Boston_data)
summary(model_ln_pred)
crPlots(model_ln_pred, col="plum3",pch=20)

**Polynomiálna - kvadratická transformácia**

Aplikujeme kvadratickú transformáciu. Model sa oproti predošlej úprave prediktora mierne zhoršil, $R^2$ poklesol na 0.2843.

In [ ]:
model_pol2 = lm(log(medv) ~ indus + I(indus^2), Boston_data)
summary(model_pol2)
crPlots(model_pol2, col="plum3")

**Polynomiálna - kubická transformácia**

Pri použití kubickej transformácie dochádza k zlepšeniu modelu a hodnota $R^2$ stúpa na 0.3203, zároveň model ešte má všetky premenné štatisticky významné.

In [ ]:
model_pol3 = lm(log(medv) ~ indus + I(indus^2) + I(indus^3), Boston_data)
summary(model_pol3)
crPlots(model_pol3, col="plum3")

**Po častiach konštantná transformácia**

Podľa zadania aplikujueme aj po častiach konstantnú transformáciu. $R^2$ značne poklesne na 0.1568.

In [ ]:
table(cut(Boston_data$indus,3))
model_cnst = lm(log(medv)~cut(indus,3), data = Boston_data)
summary(model_cnst)
crPlots(model_cnst)

**Splines**

Pre ilustráciu podľa zadania použijeme aj splines, tak aby prechádzali 1.,2.,3. kvartilom. Vrámci tejto metódy sa dostávame na $R^2=0.345$, avšak jediné významné prediktory sú intercept a jadro šiesteho stupňa. 

Nižšie vykreslíme aj preloženie dát predpoveďou a všimneme si, že v oblasti 0-5 indus dochádza k overtrainingu.

In [ ]:
install.packages("splines")
library(splines)

In [ ]:
summary(Boston_data$indus)

In [ ]:
spl_mod <- lm(log(medv) ~ bs(indus, knots = c(4,6,10)), data = Boston_data)
summary(spl_mod)


Vykreslíme si preloženie dát predpoveďovými krivkami

In [ ]:
fig(11,8)
plot(log(medv) ~ indus, data = Boston_data, col = "plum2",pch=20)
lines (sort(Boston_data$indus), sort(predict(model_ln_pred),decreasing = TRUE), col = 'violetred4', lwd = "3")
lines (sort(Boston_data$indus), sort(predict(model_pol2),decreasing = TRUE), col = 'steelblue1', lwd = "3")
lines (sort(Boston_data$indus), sort(predict(model_pol3),decreasing = TRUE), col = 'lightsalmon', lwd = "3")
lines (sort(Boston_data$indus), sort(predict(model_cnst),decreasing = TRUE), col = 'olivedrab3', lwd = "3")
lines (sort(Boston_data$indus), sort(predict(spl_mod),decreasing = TRUE), col = 'royalblue', lwd = "3")
legend( x="bottomleft", 
        legend=c("log", "pol2","pol3", "p.const","spline"),
        col=c("violetred4","steelblue1","lightsalmon","olivedrab3","royalblue"), 
        lwd=2, lty=c(1,1,1,1,1))
        

Modely možno medzi sebou porovnávať, iba ak jeden je podmodelom druhého. Môžeme teda F-testom porovnať kvadratický a kubický model. Podľa nízkych p-hodnôt vidíme, že pridanie mocnín oproti východziemu modelu tejto úlohy vedie k signifikantnému zlepšeniu modelu. to samé platí pre pridanie tretej mocniny v kubickom modeli oproti kvadratickému modelu.

In [ ]:
anova(model_pol2,model_pol3)
anova(model_ln, model_pol2,model_pol3)



* Otázka 6

  Vyberte jeden z předešlých modelů, zdůvodněte jeho výběr a validujte ho pomocí příslušných testů hypotéz na rezidua (normalita, homoscedasticita, ...) a pomocí příslušných obrázků (QQplot, residua vs. fitted, atd.)

In [ ]:
fig(11,8)
plot(log(medv) ~ indus, data = Boston_data, col = "plum2",pch=20)
lines (sort(Boston_data$indus), sort(predict(model_ln_pred),decreasing = TRUE), col = 'violetred4', lwd = "3")
lines (sort(Boston_data$indus), sort(predict(model_pol2),decreasing = TRUE), col = 'steelblue1', lwd = "3")
lines (sort(Boston_data$indus), sort(predict(model_pol3),decreasing = TRUE), col = 'lightsalmon', lwd = "3")
lines (sort(Boston_data$indus), sort(predict(model_cnst),decreasing = TRUE), col = 'olivedrab3', lwd = "3")
lines (sort(Boston_data$indus), sort(predict(spl_mod),decreasing = TRUE), col = 'royalblue', lwd = "3")
legend( x="bottomleft", 
        legend=c("log", "pol2","pol3", "p.const","spline"),
        col=c("violetred4","steelblue1","lightsalmon","olivedrab3","royalblue"), 
        lwd=2, lty=c(1,1,1,1,1))

Zobrazili sme si preloženie dát modelmi.

In [ ]:
summary(model_pol3)
crPlots(model_pol3, col="plum3",pch=20)
crPlots(lm(log(medv) ~ poly(indus,3), Boston_data), col="plum3",pch=20)

## Vícerozměrný regresní model

* Otázka 7

  Zkonstruujte lineární model s logaritmicky transformovanou odezvou `medv` a zkuste najít vztah mezi cenou a dalšími nezávislými proměnnými, které máte k dispozici (stačí aditivní model bez interakcí). Na základě kritérií jako jsou `AIC, BIC, $R^2$, F,` atd. vyberte podle vás nejvhodnější model. Lze vztah mezi `indus` a `medv`, pokud existuje, vysvětlit pomocí jiných proměnných? Tj, že například v oblastech s větším zastoupením velkoobchodu a průmyslu bydlí chudší lidé, je tam větší znečištění, nebo větší kriminalita atd.?

* Otázka 8

  Použijte ve výsledném modelu proměnnou `indus` a porovnejte jak se změnil její vliv na medián ceny nemovitostí oproti jednoduchému regresnímu modelu s log transformovanou odezvou (viz otázka 4). Jaké je snížení průměrné ceny nemovitostí při vzrůstu proměnné indus o jednu jednotku? Pokud  proměnnou `indus` v modelu nemáte tak ji pro tuto otázku do modelu přiřaďte na úkor jiné proměnné s kterou je nejvíce korelovaná.

* Otázka 9

	Prezentujte váš výsledný model pro predikci `medv`, diskutujte výsledné parametry $R^2$, $\sigma$, $F$ a porovnejte je s jednoduchým lin. modelem z otázky 6. Jak se změnily a dala se tato změna očekávat? Validujte model (jak graficky, tak pomocí příslušných testů hypotéz). Pomocí Partial regression plots a Partial residual plots diskutujte linearitu použitých proměnných.

* Otázka 10

  Na základě vašeho modelu odpovězte, zdali si myslíte, že pokud bychom dokázali snížit/zvýšit podíl maloobchodu v dané lokalitě, vedlo by to ke zvýšení cen nemovitostí určených k bydlení v dané lokalitě? 